In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress unnecessary warnings
import numpy as np
from os import listdir
import re

import sqlalchemy as sa
import cx_Oracle

from pandas.tseries.offsets import Day, MonthEnd
from dateutil.relativedelta import relativedelta
import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio.plotly import plot, iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [15]:
os.listdir()

['.ipynb_checkpoints',
 'archive.zip',
 'financial_inclusion.png',
 'findex.ipynb',
 'FINDEXCountry-Series.csv',
 'FINDEXCountry.csv',
 'FINDEXData.csv',
 'FINDEXFootNote.csv',
 'FINDEXSeries.csv',
 'wld_lmc_mmr.xlsx',
 'world_country_and_usa_states_latitude_and_longitude_values.csv']

In [32]:
lat_lon = pd.read_csv( 'world_country_and_usa_states_latitude_and_longitude_values.csv',usecols= [0,1, 2,3])
lat_lon.head(2)

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates


In [45]:
findex_country = pd.read_csv('FINDEXCountry.csv').dropna(how='all',axis='columns')
findex_country.head(1)

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,System of trade,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,General trade system,Consolidated central government,Enhanced General Data Dissemination System (e-...,1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2011",NaN,NaN,NaN,2016.0


In [63]:
#selecting the nations only
country = findex_country.dropna(subset=['Region'])[['2-alpha code','Short Name','Income Group','Region','Country Code']]
#merge with lat lon dataset on 2 alphabets country code
country = country.merge(lat_lon,left_on='2-alpha code',right_on='country_code',how='left')
#remove the columns that are duplicate and not necessary for future uses
country = country.drop(columns=['2-alpha code','Short Name','country_code']).dropna(subset=['country'])
country.head(2)

,Income Group,Region,Country Code,latitude,longitude,country
0,Low income,South Asia,AFG,33.939110,67.709953,Afghanistan
1,Lower middle income,Sub-Saharan Africa,AGO,-11.202692,17.873887,Angola


In [216]:
main.loc[(main['Country Name'].isin(['Canada','United States','New Zealand']))&(main['Indicator Name'].str.contains('Credit card'))].drop(columns=['2011','2014']).dropna(how='all',axis='columns')

,Country Name,Country Code,Indicator Name,Indicator Code,2017
38168,Canada,CAN,Credit card ownership (% age 15+),fin7.t.a,82.584816
38169,Canada,CAN,"Credit card ownership, female (% age 15+)",fin7.t.a.2,83.649445
38170,Canada,CAN,"Credit card ownership, in labor force (% age ...",fin7.t.a.10,84.012962
38171,Canada,CAN,"Credit card ownership, income, poorest 40% (%...",fin7.t.a.7,74.299095
38172,Canada,CAN,"Credit card ownership, income, richest 60% (% ...",fin7.t.a.8,88.097252
38173,Canada,CAN,"Credit card ownership, male (% age 15+)",fin7.t.a.1,81.446991
38174,Canada,CAN,"Credit card ownership, older adults (% age 25+)",fin7.t.a.4,86.535988
38175,Canada,CAN,"Credit card ownership, out of labor force (% ...",fin7.t.a.11,79.327042
38176,Canada,CAN,"Credit card ownership, primary education or le...",fin7.t.a.5,50.995872
38177,Canada,CAN,"Credit card ownership, rural (% age 15+)",fin7.t.a.9,78.415558


In [211]:
#reading main csv file
main = pd.read_csv('FINDEXData.csv')

#removing columns that have only null values
df = main.drop(columns=['Indicator Code']).dropna(how='all',axis='columns')

#removing all the rows that have only null values
df = df.set_index(['Country Name','Country Code','Indicator Name']).dropna(how='all',axis='rows').reset_index()

#taking only 2017 data
df = df.drop(columns=['Country Name','2011','2014'])

#merging with country dataset
df = df.merge(country,on='Country Code')

#making the column names uniformed
df.columns = df.columns.str.lower().str.replace(' ','_')

#split the indicator into two different groups with open parentheses as delimiter
df[['main_indicator','sub_indicator']] = df['indicator_name'].str.rsplit("(", expand=True)

#removing close parentheses
df['sub_indicator'] = df['sub_indicator'].str.replace(")",'')

#only taking main category which is % age 15+
df = df.loc[df['sub_indicator']=='% age 15+']

#removing those that contains colon or comma
df = df[~df['main_indicator'].str.contains(":")]
df = df[~df['main_indicator'].str.contains(",")]

#removing the sub indicator columns and removing those columns with null values only
df = df.drop(columns=['sub_indicator']).dropna(how='all',axis='columns')

#replacing High income: nonOECD countries as High Income
df['income_group'] = df['income_group'].replace("High income: nonOECD",'High income')
df['income_group'] = df['income_group'].str.replace(" income","")

df.head(2)

,country_code,indicator_name,2017,income_group,region,latitude,longitude,country,main_indicator
0,AFG,Account (% age 15+),14.893312,Low,South Asia,33.93911,67.709953,Afghanistan,Account
12,AFG,Borrowed any money in the past year (% age 15+),33.918972,Low,South Asia,33.93911,67.709953,Afghanistan,Borrowed any money in the past year


In [197]:
df.income_group.unique()

array(['Low', 'Upper middle', 'Lower middle', 'High'], dtype=object)

In [198]:
#Choosing the country
#country_code_list = ['MMR','AFG','GBR','OMN','USA']
country_code_list = df['country_code'].unique()
selected = df.loc[df['country_code'].isin(country_code_list)]
selected = selected.drop(columns=['indicator_name','region']).dropna(subset=['2017'])
selected['join'] = 'link'
selected.head(5)

,country_code,2017,income_group,latitude,longitude,country,main_indicator,join
0,AFG,14.893312,Low,33.93911,67.709953,Afghanistan,Account,link
12,AFG,33.918972,Low,33.93911,67.709953,Afghanistan,Borrowed any money in the past year,link
36,AFG,14.304878,Low,33.93911,67.709953,Afghanistan,Borrowed for health or medical purposes,link
48,AFG,3.313898,Low,33.93911,67.709953,Afghanistan,Borrowed from a financial institution,link
49,AFG,3.811304,Low,33.93911,67.709953,Afghanistan,Borrowed from a financial institution or used ...,link


In [199]:
padding = 36
density = pd.DataFrame(pd.Series(range(1,padding+1))).rename(columns={0:'point'})
density['join'] = 'link'
density.tail()

,point,join
31,32,link
32,33,link
33,34,link
34,35,link
35,36,link


# Account / Card ownership

In [205]:
ownership = ['Credit card ownership ','Debit card ownership ','Mobile money account ','Financial institution account ']
ownership = selected.loc[(selected['main_indicator'].isin(ownership))]
ownership['main_indicator'] = ownership['main_indicator'].str.replace(" ownership","")
ownership['2017'] = np.round(ownership['2017'],decimals=0) 
ownership[['main_indicator','country','2017']].sort_values(by=['2017'],ascending=False)
ownership.head()

,country_code,2017,income_group,latitude,longitude,country,main_indicator,join
144,AFG,1.0,Low,33.939110,67.709953,Afghanistan,Credit card,link
156,AFG,3.0,Low,33.939110,67.709953,Afghanistan,Debit card,link
169,AFG,15.0,Low,33.939110,67.709953,Afghanistan,Financial institution account,link
278,AFG,1.0,Low,33.939110,67.709953,Afghanistan,Mobile money account,link
868,ALB,8.0,Upper middle,41.153332,20.168331,Albania,Credit card,link


In [206]:
tableau = ownership.merge(density,on='join',how='outer')
tableau[['endY-latitude','endX-longitude']] = tableau[['latitude','longitude']]
tableau.head()

,country_code,2017,income_group,latitude,longitude,country,main_indicator,join,point,endY-latitude,endX-longitude
0,AFG,1.0,Low,33.93911,67.709953,Afghanistan,Credit card,link,1,33.93911,67.709953
1,AFG,1.0,Low,33.93911,67.709953,Afghanistan,Credit card,link,2,33.93911,67.709953
2,AFG,1.0,Low,33.93911,67.709953,Afghanistan,Credit card,link,3,33.93911,67.709953
3,AFG,1.0,Low,33.93911,67.709953,Afghanistan,Credit card,link,4,33.93911,67.709953
4,AFG,1.0,Low,33.93911,67.709953,Afghanistan,Credit card,link,5,33.93911,67.709953


In [207]:
tableau.loc[(tableau['point']==1)&(tableau['country'].isin(['Canada','Russia']))]

,country_code,2017,income_group,latitude,longitude,country,main_indicator,join,point,endY-latitude,endX-longitude
2700,CAN,83.0,High,56.130366,-106.346771,Canada,Credit card,link,1,56.130366,-106.346771
2736,CAN,97.0,High,56.130366,-106.346771,Canada,Debit card,link,1,56.130366,-106.346771
2772,CAN,100.0,High,56.130366,-106.346771,Canada,Financial institution account,link,1,56.130366,-106.346771
13824,RUS,20.0,Upper middle,61.524010,105.318756,Russia,Credit card,link,1,61.524010,105.318756
13860,RUS,57.0,Upper middle,61.524010,105.318756,Russia,Debit card,link,1,61.524010,105.318756
13896,RUS,76.0,Upper middle,61.524010,105.318756,Russia,Financial institution account,link,1,61.524010,105.318756


In [208]:
print(ownership.shape)
print(tableau.shape)
print(ownership.shape[0]*padding)
print(tableau.point.max())

(506, 8)
(18216, 11)
18216
36


In [209]:
tableau.to_csv('tableau.csv',index=False)
tableau.loc[tableau['country_code'].isin(['MMR','AFG','GBR','OMN','USA'])].to_csv('tableau_5_countries.csv',index=False)
tableau.loc[tableau['main_indicator'].isin(['Financial institution account '])].to_csv('tableau_1_indicator.csv',index=False)
tableau.loc[(tableau['main_indicator'].isin(['Financial institution account ']))&(tableau['country_code'].isin(['MMR','AFG','GBR','OMN','USA']))].to_csv('tableau_1_indicator_5_countries.csv',index=False)

# Digital 

In [7]:
digital = ['Used a mobile phone or the internet to access a financial institution account in the past year ',
       'Used the internet to buy something online in the past year',
       'Used the internet to pay bills in the past year ']
digital_mm = selected.loc[(selected['main_indicator'].isin(digital))&(selected['country_code']=='MMR')]
digital_mm['main_indicator'] = digital_mm['main_indicator'].str.replace('Used the internet ','')
digital_mm['main_indicator'] = digital_mm['main_indicator'].str.replace('Used a mobile phone or the internet ','')
digital_mm['2017'] = np.round(digital_mm['2017'],decimals=0) 
digital_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)

,main_indicator,2017
81898,to buy something online in the past year,3.0
81882,to access a financial institution account in t...,1.0
81910,to pay bills in the past year,1.0


In [8]:
digital_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)[['main_indicator']].values

array([['to buy something online in the past year'],
       ['to access a financial institution account in the past year '],
       ['to pay bills in the past year ']], dtype=object)

# Savings 

In [9]:
saved_mm = selected.loc[(selected['main_indicator'].str.contains('Saved'))&(selected['country_code']=='MMR')]
#saved_mm['main_indicator'] = saved_mm['main_indicator'].str.replace('No account because of','')
#saved_mm['main_indicator'] = saved_mm['main_indicator'].str.replace('saved','')
saved_mm['2017'] = np.round(saved_mm['2017'],decimals=0) 
saved_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)

,main_indicator,2017
81774,Saved any money in the past year,36.0
81810,Saved for old age,13.0
81786,Saved at a financial institution,8.0
81834,Saved using a savings club or a person outside...,8.0


In [10]:
saved_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)[['main_indicator']].values

array([['Saved any money in the past year '],
       ['Saved for old age '],
       ['Saved at a financial institution '],
       ['Saved using a savings club or a person outside the family ']],
      dtype=object)

# Borrowing 

In [11]:
borrowed_mm = selected.loc[(selected['main_indicator'].str.contains('Borrowed'))&(selected['country_code']=='MMR')]
#borrowed_mm['main_indicator'] = borrowed_mm['main_indicator'].str.replace('No account because of','')
borrowed_mm['main_indicator'] = borrowed_mm['main_indicator'].str.replace('Borrowed','')
borrowed_mm['2017'] = np.round(borrowed_mm['2017'],decimals=0) 
borrowed_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)

,main_indicator,2017
81241,any money in the past year,44.0
81325,from family or friends,22.0
81277,from a financial institution,19.0
81278,from a financial institution or used a credit...,19.0
81265,for health or medical purposes,7.0
81301,from a savings club,1.0


In [12]:
no_account_mm = selected.loc[(selected['main_indicator'].str.contains('No account'))&(selected['country_code']=='MMR')]
no_account_mm['main_indicator'] = no_account_mm['main_indicator'].str.replace('No account because of','')
no_account_mm['main_indicator'] = no_account_mm['main_indicator'].str.replace('No account because','')
no_account_mm['2017'] = np.round(no_account_mm['2017'],decimals=0) 
no_account_mm[['main_indicator','2017']].sort_values(by=['2017'],ascending=False)

,main_indicator,2017
81525,insufficient funds,57.0
81527,lack of necessary documentation,24.0
81521,financial institutions are too far away,17.0
81523,financial services are too expensive,7.0
81535,someone in the family has an account,7.0
81531,no need for financial services ONLY,5.0
81533,religious reasons,2.0
81529,lack of trust in financial institutions,1.0


# Appendix

In [13]:
a.shape()

NameError: name 'a' is not defined

In [ ]:
sunburst = df.loc[(df['region']=='East Asia & Pacific')]
sunburst = sunburst.loc[(sunburst['income_group']=='Lower middle income')]
ownership = ['Credit card ownership ','Debit card ownership ','Mobile money account ','Financial institution account ']
sunburst = sunburst.loc[sunburst['main_indicator'].isin(ownership)]
sunburst = sunburst.drop(columns=['indicator_name']).dropna(subset=['2017'])
sunburst.head()

In [ ]:
fig = px.sunburst(sunburst, path=['region', 'income_group', 'short_name','main_indicator'], values='2017')
fig.show()

In [ ]:
#df.loc[df['income_group'].isnull()][['country_code','short_name']].drop_duplicates()
#df.loc[df['income_group']=='Lower middle income'].short_name.unique()
#df.loc[df['income_group']=='Low income'].short_name.unique()